<a href="https://colab.research.google.com/github/Naman-27072004/RockShield---AI-Copyright-Code-/blob/main/RockShield_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
#   LANDSLIDE RISK ML MODEL - UNIFIED STRUCTURE
#   Works for: Colab, Local, Django, API, Batch Prediction
# ======================================================

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os

# ======================================================
#   CONFIG
# ======================================================
MODEL_PATH = "rf_landslide_classifier.pkl"
CSV_PATH = "landslide_merged_demo.csv"      # put your dataset here

TARGET = "label"

class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

# ======================================================
#   1. TRAIN THE MODEL
# ======================================================
def train_model(csv_path=CSV_PATH, save=True):
    print("📌 Loading CSV:", csv_path)
    df = pd.read_csv(csv_path)

    X = df.drop(columns=["timestamp", TARGET])
    y = df[TARGET]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("model", RandomForestClassifier(
            n_estimators=200,
            max_depth=12,
            random_state=42,
            class_weight="balanced"
        ))
    ])

    pipeline.fit(X_train, y_train)

    print("\n🎯 ACCURACY")
    print("Train Accuracy:", pipeline.score(X_train, y_train))
    print("Test Accuracy :", pipeline.score(X_test, y_test))

    if save:
        joblib.dump(pipeline, MODEL_PATH)
        print(f"\n💾 Model saved as {MODEL_PATH}")

    return pipeline, X.columns.tolist()


# ======================================================
#   2. LOAD SAVED MODEL
# ======================================================
def load_model(model_path=MODEL_PATH):
    if not os.path.exists(model_path):
        raise FileNotFoundError("❌ Model not found. Train it first.")

    pipeline = joblib.load(model_path)
    feature_names = pipeline.named_steps["imputer"].feature_names_in_
    print("✅ Model loaded successfully")
    return pipeline, feature_names


# ======================================================
#   3. SINGLE PREDICTION HELPER
# ======================================================
def predict_risk(input_dict, pipeline=None, feature_names=None):

    if pipeline is None:
        pipeline, feature_names = load_model()

    df_input = pd.DataFrame([input_dict])

    # Ensure missing columns are added
    for col in feature_names:
        if col not in df_input:
            df_input[col] = np.nan

    df_input = df_input[feature_names]  # maintain order

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}

    return pred_label, proba_dict


# ======================================================
#   4. OPTIONAL: QUICK TEST
# ======================================================
if __name__ == "__main__":
    # Train the model (only run once)
    pipeline, features = train_model()

    # Example input
    example_input = {
        "temperature_C": 14.868,
        "rainfall_mm": 6.946,
        "pitch_deg": 2.4122,
        "soil_moisture": 0.22,
        "roll_deg": 0.9999,
        "crack_mm": 0.49916,
        "crack_growth_mm_per_hr": 0.00017,
        "accel_x_g": -0.03961,
        "accel_y_g": 0.01297,
        "accel_z_g": 0.99266,
        "vibration_rms_g": 0.04279,
        "vibration_peak_g": 0.02472
    }

    pred, proba = predict_risk(example_input, pipeline, features)
    print("\n🔮 Prediction:", pred)
    print("📊 Probabilities:", proba)


📌 Loading CSV: landslide_merged_demo.csv

🎯 ACCURACY
Train Accuracy: 0.91875
Test Accuracy : 0.87525

💾 Model saved as rf_landslide_classifier.pkl

🔮 Prediction: High Risk
📊 Probabilities: {'Stable': 0.1874943912054663, 'High Risk': 0.6624961462947841, 'Landslide Event': 0.15000946249974956}


This code defines a REST API endpoint (/predict) for predicting landslide risk levels based on environmental sensor readings — such as temperature, rainfall, and soil moisture.
It trains a Random Forest machine learning model automatically when the server starts, then uses it to make predictions for incoming POST requests.(This is Not a colab code)

In [ ]:
# # views.py

# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# import os

# # --- Load CSV and train model at startup ---
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# CSV_PATH = os.path.join(BASE_DIR, "landslide_merged_demo.csv")

# # Load dataset
# df = pd.read_csv(CSV_PATH)

# # Define target and features
# TARGET = "label"
# X = df.drop(columns=["timestamp", TARGET])
# y = df[TARGET]

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # ML Pipeline: Imputation → Scaling → RandomForest
# pipeline = Pipeline([
#     ("imputer", SimpleImputer(strategy="mean")),
#     ("scaler", StandardScaler()),
#     ("model", RandomForestClassifier(
#         n_estimators=200,
#         max_depth=12,
#         random_state=42,
#         class_weight="balanced"
#     ))
# ])

# # Train model once at server startup
# pipeline.fit(X_train, y_train)

# print("✅ Model trained at startup")
# print("Train Accuracy:", pipeline.score(X_train, y_train))
# print("Test Accuracy :", pipeline.score(X_test, y_test))

# # Label mapping
# class_labels = {
#     0: "Stable",
#     1: "High Risk",
#     2: "Landslide Event"
# }

# # --- Helper: Predict risk from input JSON ---
# def predict_risk(input_dict):
#     df_input = pd.DataFrame([input_dict])

#     # Ensure all expected features exist
#     for col in X.columns:
#         if col not in df_input:
#             df_input[col] = np.nan

#     # Keep same feature order
#     df_input = df_input[X.columns]

#     # Run model
#     pred = pipeline.predict(df_input)[0]
#     proba = pipeline.predict_proba(df_input)[0]

#     # Convert to readable output
#     pred_label = class_labels[pred]
#     proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}

#     return pred_label, proba_dict

# # --- REST API Endpoint ---
# @api_view(['POST'])
# def predict(request):
#     """
#     POST /predict
#     Input: JSON with feature values
#     Output: Predicted risk + probabilities
#     """
#     input_data = request.data  # Expect JSON payload
#     prediction, probabilities = predict_risk(input_data)

#     return Response({
#         "prediction": prediction,
#         "probabilities": probabilities
#     })


curl -X POST http://voidpanel.com/predict/ \
     -H "Content-Type: application/json" \
     -d '{
        "temperature_C": 14.868,
        "rainfall_mm": 6.946,
        "pitch_deg": 2.4122,
        "soil_moisture": 0.22,
        "roll_deg": 0.9999,
        "crack_mm": 0.49916,
        "crack_growth_mm_per_hr": 0.00017,
        "accel_x_g": -0.03961,
        "accel_y_g": 0.01297,
        "accel_z_g": 0.99266,
        "vibration_rms_g": 0.04279,
        "vibration_peak_g": 0.02472
     }'
